In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE



In [18]:
def rfeFeature(indep_x, dep_y):
    rfelist=[]
    
    from sklearn.linear_model import LinearRegression
    lin=LinearRegression()
    
    from sklearn.svm import SVR
    svr=SVR(kernel='linear')
    
    from sklearn.tree import DecisionTreeRegressor
    dec = DecisionTreeRegressor(random_state = 0)
    
    from sklearn.ensemble import RandomForestRegressor
    rf = RandomForestRegressor(n_estimators = 10, random_state = 0)

    modellist=[lin,svr,dec,rf]

    for i in modellist:
        rfe=RFE(i,n_features_to_select=7)
        fit1=rfe.fit(indep_x,dep_y)
        test=fit1.transform(indep_x)
        rfelist.append(test)
    return rfelist
        
        
    

In [21]:
def split_scaler(indep_x,dep_y):
        x_train, x_test, y_train, y_test = train_test_split(indep_x, dep_y, test_size = 0.25, random_state = 0)
        sc = StandardScaler()
        x_train = sc.fit_transform(x_train)
        x_test = sc.transform(x_test)    
        return x_train, x_test, y_train, y_test

In [22]:
def r2_prediction(regressor,x_test,y_test):
    y_pred=regressor.predict(x_test)
    from sklearn.metrics import r2_score
    r2score=r2_score(y_test,y_pred)
    return r2score
    
    
    

In [33]:
def Linear(x_train,y_train,x_test,y_test):       
    from sklearn.linear_model import LinearRegression
    regressor = LinearRegression()
    regressor.fit(x_train, y_train)
    r2=r2_prediction(regressor,x_test,y_test)
    return  r2   

def svm(x_train,y_train,x_test,y_test):       
    from sklearn.svm import SVR
    regressor = SVR(kernel = 'linear')
    regressor.fit(x_train, y_train)
    r2=r2_prediction(regressor,x_test,y_test)
    return  r2 

def dtree(x_train,y_train,x_test,y_test):       
    from sklearn.tree import DecisionTreeRegressor
    regressor = DecisionTreeRegressor(random_state = 0)
    regressor.fit(x_train, y_train)
    r2=r2_prediction(regressor,x_test,y_test)
    return  r2 

def rforest(x_train,y_train,x_test,y_test):       
    from sklearn.ensemble import RandomForestRegressor
    regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
    regressor.fit(x_train, y_train)
    r2=r2_prediction(regressor,x_test,y_test)
    return  r2 

def rfeRegression(acclin,accsvml,accdes,accrf):
    rfe_table=pd.DataFrame(index=['linear','svm','DTree','RForest'],columns=['linear','svm','DTree','RForest'])
    
    for number,idex in enumerate(rfe_table.index):
        rfe_table['linear'][idex]=acclin[number]
        rfe_table['svm'][idex]=accsvml[number]
        rfe_table['DTree'][idex]=accdes[number]
        rfe_table['RForest'][idex]=accrf[number]
    return rfe_table
      
       
    

In [34]:
dataset=pd.read_csv('insurance_pre.csv')
df=dataset

In [35]:
df=pd.get_dummies(df,drop_first=True)

In [36]:
df.head()

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,False,True
1,18,33.770,1,1725.55230,True,False
2,28,33.000,3,4449.46200,True,False
3,33,22.705,0,21984.47061,True,False
4,32,28.880,0,3866.85520,True,False


In [37]:
indep_x=df.drop('charges',axis=1)
dep_y=df['charges']

In [38]:
indep_x.head()

,age,bmi,children,sex_male,smoker_yes
0,19,27.900,0,False,True
1,18,33.770,1,True,False
2,28,33.000,3,True,False
3,33,22.705,0,True,False
4,32,28.880,0,True,False


In [39]:
rfeVar=rfeFeature(indep_x, dep_y)

r2lin=[]
r2svm=[]
r2des=[]
r2rf=[]

for i in rfeVar:
    x_train, x_test, y_train, y_test=split_scaler(i,dep_y) 
    r2l= Linear(x_train,y_train,x_test,y_test)
    r2lin.append(r2l)

    r2s= svm(x_train,y_train,x_test,y_test)
    r2svm.append(r2s)

    r2d=dtree(x_train,y_train,x_test,y_test)
    r2des.append(r2s)

    r2r=rforest(x_train,y_train,x_test,y_test)
    r2rf.append(r2r)


    

In [40]:
result= rfeRegression(r2lin,r2svm,r2des,r2rf)

In [41]:
result

,linear,svm,DTree,RForest
linear,0.795217,-0.011016,-0.011016,0.85246
svm,0.795217,-0.011016,-0.011016,0.85246
DTree,0.795217,-0.011016,-0.011016,0.85246
RForest,0.795217,-0.011016,-0.011016,0.85246
